In [3]:
#imports
import pandas as pd
import gc
import re
import numpy as np

import warnings 
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
import torch
import datasets
import transformers
print(transformers.__version__)

4.28.1


In [4]:
RATE_HZ = 16000 # resampling rate in Hz
MAX_LENGTH = 160000 # maximum audio interval length to consider (= RATE_HZ * SECONDS)
labels = ['cat', 'dog']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

{0: 'cat', 1: 'dog'} 

 {'cat': 0, 'dog': 1}


In [5]:
import torchaudio
print(torchaudio.__version__)

0.12.0+cu113


In [8]:
from pathlib import Path

lang_counter = dict()

def load_data():
    file_list = []
    label_list = []
    for file in Path('D:\ML\cats_dogs').glob('*/*/*.wav'):
        lang = re.match(r"([a-z]+)", file.stem).group(1)
        file_list.append(file)
        label_list.append(label2id[lang])
    dd = pd.DataFrame()
    dd['file'] = file_list
    dd['label'] = label_list
    return dd

In [9]:
%%time
dd = load_data()
print(dd)

                                             file  label
0           D:\ML\cats_dogs\test\cats\cat_110.wav      0
1           D:\ML\cats_dogs\test\cats\cat_112.wav      0
2           D:\ML\cats_dogs\test\cats\cat_115.wav      0
3           D:\ML\cats_dogs\test\cats\cat_126.wav      0
4           D:\ML\cats_dogs\test\cats\cat_129.wav      0
..                                            ...    ...
272  D:\ML\cats_dogs\train\dog\dog_barking_94.wav      1
273  D:\ML\cats_dogs\train\dog\dog_barking_95.wav      1
274  D:\ML\cats_dogs\train\dog\dog_barking_96.wav      1
275  D:\ML\cats_dogs\train\dog\dog_barking_97.wav      1
276  D:\ML\cats_dogs\train\dog\dog_barking_98.wav      1

[277 rows x 2 columns]
Wall time: 17.3 ms
